In [1]:
import os
import glob
import pandas as pd
from datasets import load_dataset, load_from_disk, Dataset, ClassLabel

## Pushed to HuggingFace


In [ ]:
# Raw Dataset

# Convert csv to Apache Arrow format
raw_git_commits = load_dataset("csv", data_files="./data/commits.csv")
raw_git_commits.save_to_disk("./data/raw-git-commits")

# Upload to HuggingFace
raw_git_commits = load_from_disk("./data/raw-git-commits")
raw_git_commits.push_to_hub("neuralsentry/raw-git-commits")

In [ ]:
# Split Dataset

raw_git_commits = load_dataset("neuralsentry/raw-git-commits")
git_commits = raw_git_commits["train"].train_test_split(test_size=0.1, seed=420)

# Upload to HuggingFace
git_commits.push_to_hub("neuralsentry/git-commits")

In [ ]:
# Labelled Dataset

# Convert all excels to csv
labelled_excels = glob.glob("./data/labelled/*.xlsx")
for excel in labelled_excels:
    df = pd.read_excel(excel)
    df.to_csv(excel.replace(".xlsx", ".csv"), index=False)

# Combine all csvs into one dataset
labelled_csvs = glob.glob("./data/labelled/*.csv")
df = pd.concat([pd.read_csv(csv) for csv in labelled_csvs], ignore_index=True)
Dataset.from_pandas(df).save_to_disk("./data/git-commits-labelled")
git_commits_labelled = load_from_disk("./data/git-commits-labelled")

# Change typeof labels to ClassLabel
features = git_commits_labelled.features.copy()
features["labels"] = ClassLabel(num_classes=2, names=["non-bug", "bug"])
git_commits_labelled = git_commits_labelled.cast(features)

# Push
git_commits_labelled.push_to_hub("neuralsentry/git-commits-labelled")

## Data Labelling


In [ ]:
raw_datasets = load_dataset("neuralsentry/raw-git-commits", split="train").shuffle(seed=420)

In [ ]:
# Extract commits of each repo

remote_urls = raw_datasets.unique("remote_url")
repo_names = [os.path.basename(url) for url in remote_urls]

for repo_name in repo_names:
    raw_datasets.filter(lambda example: repo_name in example["remote_url"]).to_csv(
        f"./data/{repo_name}.csv"
    )

In [ ]:
# Remove already labelled commits
for repo_name in repo_names:
    path = f"./data/{repo_name}.csv"
    labelled_path = f"./data/labelled/{repo_name}.csv"
    if not os.path.exists(labelled_path):
        continue
    unlabelled_df = pd.read_csv(path)
    labelled_df = pd.read_csv(labelled_path)

    # remove all rows in unlabelled_df where the sha is in labelled_df
    df = unlabelled_df[~unlabelled_df["sha"].isin(labelled_df["sha"])]
    df.to_csv(path, index=False)